<a href="https://colab.research.google.com/github/boothmanrylan/canadaMSSForestDisturbances/blob/main/buildMaps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!git clone https://github.com/boothmanrylan/canadaMSSForestDisturbances.git
%cd canadaMSSForestDisturbances

In [ ]:
!pip install -q -q --upgrade pip
!pip install -q -q "apache-beam[gcp]==2.50.0"
!pip install -q -q msslib
!pip install -q rasterio

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import io
import itertools

import google
from google.colab import auth
from google.api_core import retry

import requests

import ee
import geemap
import geopandas

import numpy as np
from numpy.lib import recfunctions as rfn
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt

from msslib import msslib
from mss_forest_disturbances import constants

In [ ]:
auth.authenticate_user()

os.environ['GOOGLE_CLOUD_PROJECT'] = constants.PROJECT
!gcloud config set project {constants.PROJECT}

credentials, _ = google.auth.default()
ee.Initialize(
    credentials,
    project=constants.PROJECT,
    opt_url=constants.HIGH_VOLUME_ENDPOINT
)

In [ ]:
temp_location = os.path.join(constants.BUCKET, 'temp')
staging_location = os.path.join(constants.BUCKET, 'staging')

input_asset = os.path.join(constants.ASSET_PATH, 'data', 'model2', 'train')
output_prefix = os.path.join(constants.BUCKET, 'scratch', 'test_build_maps', 'final_output')
model_checkpoint_path = os.path.join(constants.BUCKET, 'scratch', 'models', 'test2', 'checkpoints')
model_one_output_prefix = os.path.join(constants.BUCKET, 'scratch', 'test_build_maps', 'model_one_output')
start_year = 1985
end_year = 1990

!python build_maps.py \
    --runner="DirectRunner" \
    --project={constants.PROJECT} \
    --job_name='test-build-maps' \
    --region='us-central1' \
    --max-requests=20 \
    --input-asset={input_asset} \
    --output-prefix={output_prefix} \
    --model-checkpoint-path={model_checkpoint_path} \
    --model-one-output-prefix={model_one_output_prefix} \
    --start-year={start_year} \
    --end-year={end_year} \
    # --experiments=use_runner_v2 \
    # --sdk_container_image={constants.DOCKER_IMAGE_URI} \
    # --sdk_location=container \
    # --temp_location={temp_location} \
    # --staging_location={staging_location} \